# Scraping

In [1]:
from sre_constants import NOT_LITERAL
from urllib.parse import uses_params
import requests
import pandas as pd
import praw
import re
import credentials

In [2]:
#inputs
client_id=credentials.client_id
client_secret=credentials.client_secret
user_agent="MyBot/0.0.1"
password=credentials.reddit_password
username=credentials.reddit_username
search = "adidas"
search_limit = 15
sort = "relevance"

In [16]:
'''_________FUNCTIONS__________'''

def upvote_ratio(ups, downs):
    total = ups + downs
    if total >0:
        ratio = float(ups/total)
        return ratio
    else:
        return 0

def search_string(search):
    result = search.replace(' ', '%20') 
    return result

def clean_text(text):                               # user defined function for cleaning text
    text = text.lower()                             # all lower case
    text = re.sub(r'\[.*?\]', ' ', text)            # remove text within [ ] (' ' instead of '')
    text = re.sub(r'\<.*?\>', ' ', text)            # remove text within < > (' ' instead of '')
    text = re.sub(r'http\S+', ' ', text)            # remove website ref http
    text = re.sub(r'www\S+', ' ', text)             # remove website ref www

    text = text.replace('€', 'euros')               # replace special character with words
    text = text.replace('£', 'gbp')                 # replace special character with words
    text = text.replace('$', 'dollar')              # replace special character with words
    text = text.replace('%', 'percent')             # replace special character with words
    text = text.replace('\n', ' ')                  # remove \n in text that has it

    text = text.replace('\'', '’')                  # standardise apostrophe
    text = text.replace('‚Äô;', '’')                # standardise apostrophe
    text = text.replace('‚Äù', '’')                 # standardise apostrophe
    text = text.replace('Äú', '’')                  # standardise apostrophe
    text = text.replace('‚Ä¶', '')                  # standardise apostrophe

    text = text.replace('’d', ' would')             # remove ’ (for would, should? could? had + PP?)
    text = text.replace('’s', ' is')                # remove ’ (for is, John's + N?)
    text = text.replace('’re', ' are')              # remove ’ (for are)
    text = text.replace('’ll', ' will')             # remove ’ (for will)
    text = text.replace('’ve', ' have')             # remove ’ (for have)
    text = text.replace('’m', ' am')                # remove ’ (for am)
    text = text.replace('can’t', 'can not')         # remove ’ (for can't)
    text = text.replace('won’t', 'will not')        # remove ’ (for won't)
    text = text.replace('n’t', ' not')              # remove ’ (for don't, doesn't)

    text = text.replace('’', ' ')                   # remove apostrophe (in general)
    text = text.replace('&quot;', ' ')              # remove quotation sign (in general)

    text = text.replace('cant', 'can not')          # typo 'can't' (note that cant is a proper word)
    text = text.replace('dont', 'do not')           # typo 'don't'

    text = re.sub(r'[^a-zA-Z0-9]', r' ', text)      # only alphanumeric left
    text = text.replace("   ", ' ')                 # remove triple empty space
    text = text.replace("  ", ' ')                  # remove double empty space
    return text


In [17]:
'''_________RAW REDDIT API CODE__________'''

auth = requests.auth.HTTPBasicAuth(client_id, client_secret)

data = {'grant_type': 'password',
        'username': username,
        'password': password}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': user_agent}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers) 

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests

res = requests.get('https://oauth.reddit.com/search/?q='+ search_string(search) +'&sort=' + sort, headers=headers,  params = {'limit': str(search_limit)}) ##search function --> automate!!
# res = requests.get('https://oauth.reddit.com/r/cars/hot', headers=headers,  params = {'limit': str(search_limit)}) ##search function --> automate!!
#print(res.json()) #--> uncomment if you wanna see what is in the link
#print(post["data"].keys()) #--> uncomment to see the keys you can use for post['data']
#post['kind']+'_'+ post['data']['id'] #--> get the unique id for the post

In [18]:
'''_________USING PRAW LIBRARY FOR COMMENTS__________'''

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent,
    password=password,
    username=username,
)

In [19]:
'''_________PANDAS__________'''
df = pd.DataFrame()


for post in res.json()['data']['children']:
    print(post['data']['id'])


for post in res.json()['data']['children']:
    df = df.append({
        'subreddit': clean_text(post['data']['subreddit']),
        'title': clean_text(post['data']['title']),
        'selftext': clean_text(post['data']['selftext']),
        'ups': post['data']['ups'],
        # 'downs': post['data']['downs'],
        # 'upvote_ratio': post['data']['upvote_ratio'],
        'overall_text': clean_text(post['data']['title']) + '. ' + clean_text(post['data']['selftext']),
    }, ignore_index=True)

    submission = reddit.submission(url="https://www.reddit.com/" + post['data']['id'])
    comments = submission.comments.replace_more(limit=0)

    submission.comments.replace_more(limit = 0)

    for comment in submission.comments.list(): #NOT INCLUDING COMMENT REPLIES
        df = df.append({ 
            # 'comment': clean_text(comment.body),
            'comment ups': comment.ups,
            # 'comment downs': comment.downs,
            'overall_text': clean_text(comment.body),
            #'comment upvote_ratio': comment.upvote_ratio(comment.ups, comment.downs),
            }, ignore_index=True)

osdz6x
17tnjxs
175l6fy
12fawr0
1644uga
13m1rte
16r2nlg
y6xnfz
15c5gvr
w2yct0
683334
a0vj09
yci78s
12g4cg6
k68u5z


<ipython-input-19-627f8e180bec>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-19-627f8e180bec>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-19-627f8e180bec>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-19-627f8e180bec>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-19-627f8e180bec>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
<ipython-input-19-627f8e180bec>:26: FutureWarning: The frame

In [20]:
print(df)

        subreddit                                           title selftext  \
0     fashionreps   adidas tracksuit from the adidas secret store            
1             NaN                                             NaN      NaN   
2             NaN                                             NaN      NaN   
3             NaN                                             NaN      NaN   
4             NaN                                             NaN      NaN   
...           ...                                             ...      ...   
1781          NaN                                             NaN      NaN   
1782          NaN                                             NaN      NaN   
1783          NaN                                             NaN      NaN   
1784          NaN                                             NaN      NaN   
1785          NaN                                             NaN      NaN   

       ups                                       overall_text  

In [21]:
# remove 'overall text' entries that are empty
df = df[df['overall_text'] != ' ']

# Sentiment Analysis

In [22]:
# sentiment analysis using HuggingFace
from transformers import pipeline
sentiment_classifier = pipeline('sentiment-analysis')

def get_sentiment(text):
    # Truncate the text to 512 tokens
    truncated_text = text[:512]
    # Perform sentiment analysis
    sentiment_label = sentiment_classifier(truncated_text)[0]['label']
    return sentiment_label


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [23]:
get_sentiment('trash ecosystem')

'NEGATIVE'

In [24]:
df['sentiment'] = df['overall_text'].apply(get_sentiment)

In [25]:
df.head(100)

,subreddit,title,selftext,ups,overall_text,comment ups,sentiment
0,fashionreps,adidas tracksuit from the adidas secret store,,88.0,adidas tracksuit from the adidas secret store.,NaN,NEGATIVE
1,NaN,NaN,NaN,NaN,you top has leaf logo and pants have normal logo,18.0,POSITIVE
2,NaN,NaN,NaN,NaN,how do you even order from him,6.0,NEGATIVE
3,NaN,NaN,NaN,NaN,w2c top bottom link to the secret store yupoo,4.0,NEGATIVE
4,NaN,NaN,NaN,NaN,do you like hardbass,9.0,POSITIVE
...,...,...,...,...,...,...,...
102,NaN,NaN,NaN,NaN,how it fit,1.0,POSITIVE
103,NaN,NaN,NaN,NaN,i just got these i initially was going for the...,1.0,NEGATIVE
104,NaN,NaN,NaN,NaN,it is the comfort for me,1.0,POSITIVE
107,NaN,NaN,NaN,NaN,3 7 days normally for everything if not hit yo...,2.0,NEGATIVE


In [26]:
# count the number of positive, negative and neutral sentiments
df['sentiment'].value_counts()

NEGATIVE    1103
POSITIVE     591
Name: sentiment, dtype: int64

In [27]:
# #save to csv
# df.to_csv(r'/Users/raymondharrison/Desktop/DND/{}.csv'.format(search))

In [28]:
#drop all negative sentiments
df = df[df.sentiment != 'NEGATIVE']

print(len(df))

591


# BERT (Relevance)

In [29]:
# zero shot classification for positive sentiment

from transformers import pipeline
classifier = pipeline("zero-shot-classification")

def get_relevance(text):
    relevance = [f'related to {search}', f'not related to {search}']
    result = classifier(text, relevance)
    return result['labels'][0]



No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


In [30]:
get_relevance('he seems so happy with the toilet')

'related to adidas'

In [31]:
get_relevance('the new iphone is so clean')

'not related to adidas'

In [32]:
df['relevance'] = df['overall_text'].apply(get_relevance)

In [33]:
#count related and not related
df['relevance'].value_counts()

related to adidas        436
not related to adidas    155
Name: relevance, dtype: int64

In [34]:
# remove entries that are not relevant
df = df[df.relevance != f'not related to {search}']

In [35]:
len(df)

436

In [36]:
df.head(10)

,subreddit,title,selftext,ups,overall_text,comment ups,sentiment,relevance
1,NaN,NaN,NaN,NaN,you top has leaf logo and pants have normal logo,18.0,POSITIVE,related to adidas
4,NaN,NaN,NaN,NaN,do you like hardbass,9.0,POSITIVE,related to adidas
9,NaN,NaN,NaN,NaN,u repknowledge please approve,1.0,POSITIVE,related to adidas
15,NaN,NaN,NaN,NaN,remind me 3 day,1.0,POSITIVE,related to adidas
17,NaN,NaN,NaN,NaN,link,1.0,POSITIVE,related to adidas
26,NaN,NaN,NaN,NaN,did you size up once or what,1.0,POSITIVE,related to adidas
29,NaN,NaN,NaN,NaN,i really love the towelie 80s please tell me t...,11.0,POSITIVE,related to adidas
31,NaN,NaN,NaN,NaN,link to those last camupuses,3.0,POSITIVE,related to adidas
32,NaN,NaN,NaN,NaN,link to the last ones,2.0,POSITIVE,related to adidas
37,NaN,NaN,NaN,NaN,whats the link for towlie,1.0,POSITIVE,related to adidas


In [37]:
# #save to csv
# df.to_csv(r'/Users/raymondharrison/Desktop/DND/{}.csv'.format(search))

# GPT API Semantic mapping

In [439]:
name = "daiso copy"
df = pd.read_csv(f'/Users/raymondharrison/Desktop/DND/{name}.csv')

In [440]:
# # Define the bags of words for each label
# labels_bags = {
#     'Aggressive': ['character', 'savage', 'fun', 'hard', 'menacing', 'quick', 'rough', 'punishing', 'muscular', 'bold', 'animalistic', 'intense', 'fierce', 'assertive', 'dominant', 'forceful', 'combative', 'vigorous', 'proactive', 'unyielding', 'ferocious', 'hostile', 'agitated', 'confrontational', 'relentless', 'aggressive'],
#     'Dynamic': ['noise', 'drama', 'performance', 'spacious', 'comfort', 'cramped', 'smoothed', 'marmite', 'unpredictable', 'dramatic', 'response', 'entertaining', 'exuberant', 'agile', 'energetic', 'vibrant', 'spirited', 'active', 'lively', 'flexible', 'adaptive', 'variable', 'kinetic', 'vivacious', 'animated', 'changing', 'fluid', 'stimulating', 'dynamic'],
#     'Elegant': ['pretty', 'refinement', 'luxurious', 'relaxing', 'opulent', 'quality', 'quiet', 'beautiful', 'clever', 'gorgeously', 'proper', 'refined', 'roomy', 'posh','exquisite', 'handsome', 'detail', 'elevated', 'sophisticated', 'stylish', 'graceful', 'cultured', 'polished', 'chic', 'tasteful', 'svelte', 'dignified', 'classic', 'delicate', 'ornate', 'stately', 'aesthetic', 'elegant'],
#     'Friendly': ['comfortable', 'cheap', 'cheerful', 'practical', 'endearing', 'honest', 'easy', 'comfy', 'roomy', 'practicality', 'economy', 'comfort', 'performance', 'quietness', 'soothing', 'pleasant', 'quiet', 'welcoming', 'warm', 'genial', 'amicable', 'cordial', 'affable', 'convivial', 'benign', 'accessible', 'sociable', 'gracious', 'harmonious', 'neighborly', 'engaging', 'friendly', 'affordable'],
#     'Modern': ['new', 'practical', 'uncomplicated', 'uncompromising', 'improved', 'smarter', 'efficient','comfortable', 'contemporary', 'current', 'trendy', 'futuristic', 'progressive', 'up-to-date', 'state-of-the-art', 'cutting-edge', 'innovative', 'avant-garde', 'new-age', 'sleek', 'novel', 'modern'],
#     'Powerful': ['light', 'hard', 'frenetic', 'strong', 'potent', 'commanding', 'dynamic', 'influential', 'sturdy', 'robust', 'intense', 'authoritative', 'mighty', 'dominant', 'compelling', 'forceful', 'resolute', 'powerful'],
#     'Sporty': ['manual', 'muscle', 'fast', 'performance', 'practical', 'big', 'small', 'athletic', 'nippy', 'zippy', 'energetic', 'lively', 'peppy', 'vigorous', 'racy', 'competitive', 'active', 'sprightly', 'fit', 'agile', 'spirited', 'sporty'],
#     'Stable': ['safe', 'firm', 'built', 'solidly', 'tidy', 'tough', 'roomy', 'practical', 'dynamic', 'reliable', 'boxy', 'commanding', 'heavy', 'wide', 'secure', 'steady', 'constant', 'dependable', 'sturdy', 'solid', 'immovable', 'established', 'fixed', 'durable', 'trustworthy', 'anchored', 'resilient', 'consistent', 'stable']
# }

In [441]:
# new bag of words

labels_bags = {
    'Aggressive': ['character', 'savage', 'hard', 'menacing', 'rough', 'punishing', 'muscular', 'bold', 'animalistic', 'intense', 'forceful', 'combative', 'vigorous', 'unyielding', 'ferocious', 'relentless', 'aggressive'],
    'Dynamic': ['performance',  'smoothed', 'unpredictable', 'dramatic', 'responsive', 'exuberant', 'agile', 'energetic', 'vibrant', 'active', 'lively', 'flexible', 'adaptive', 'variable', 'kinetic', 'vivacious', 'animated', 'changing', 'fluid', 'bold', 'dynamic'],
    'Elegant': ['refinement', 'luxurious', 'opulent', 'quality', 'quiet', 'beautiful', 'gorgeous', 'refined', 'posh', 'exquisite', 'sophisticated', 'stylish', 'graceful', 'cultured' ,'chic', 'tasteful', 'dignified', 'classic', 'delicate',  'stately', 'aesthetic', 'elegant'],
    'Friendly': ['comfort', 'cheap', 'cheerful', 'practical', 'endearing', 'easy',  'practicality', 'economy',  'soothing', 'pleasant', 'quiet', 'welcoming', 'warm', 'genial', 'amicable', 'cordial', 'affable', 'accessible', 'sociable', 'gracious', 'harmonious', 'friendly', 'affordable'],
    'Modern': ['new', 'practical', 'uncomplicated', 'uncompromising', 'improved', 'smarter', 'efficient','comfortable', 'contemporary', 'current', 'trendy', 'futuristic', 'progressive', 'state-of-the-art', 'cutting-edge', 'innovative',  'sleek', 'novel', 'modern'],
    'Powerful': ['light', 'hard', 'frenetic', 'strong', 'potent', 'commanding',  'influential', 'sturdy', 'robust', 'intense', 'authoritative', 'mighty', 'dominant', 'compelling', 'forceful', 'resolute', 'performance', 'powerful'],
    'Sporty': ['manual', 'muscle', 'fast', 'performance', 'practical', 'big', 'small', 'athletic', 'nippy', 'zippy', 'energetic', 'lively', 'peppy', 'vigorous', 'racy', 'competitive', 'active', 'sprightly', 'fit', 'agile', 'spirited', 'sporty'],
    'Stable': ['safe', 'firm', 'built', 'solidly', 'practical', 'reliable', 'boxy', 'heavy', 'wide', 'secure', 'steady', 'constant', 'dependable', 'sturdy', 'solid', 'immovable', 'established', 'fixed', 'durable', 'trustworthy', 'anchored', 'resilient', 'consistent', 'stable']
}

In [442]:
keys_list = list(labels_bags.keys())
keys_string = ', '.join(keys_list)
print(keys_string)


Aggressive, Dynamic, Elegant, Friendly, Modern, Powerful, Sporty, Stable


In [443]:
# https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api

import openai
import time

# Set OpenAI API key
openai.api_key = credentials.api_key

def GPTextract(prompt, model="gpt-3.5-turbo-1106", labels_bags=labels_bags):

    retries = 3    
    while retries > 0:    
        try: 
            messages = [
                {"role": "system", "content": 
                f'''

                You are a powerful data analyst looking at labeling unstructured text semantically according to my bag of words. 
                {str(labels_bags)}

                You must output only one word from the bag of words(choose from {keys_string}). Dont need to give me an explanation.

                '''},
                {"role": "user", "content": prompt}
                ]
            response = openai.ChatCompletion.create(
                model=model, # gpt-3.5-turbo
                messages=messages, # prompt
                temperature=0, # temperature 0 means deterministic, 1 means random
            )
            text = response.choices[0].message["content"]
            text = text.replace("\n","")
            return text
        except Exception as e:    
            if e: 
                print(e)   
                print('Timeout error, retrying...')    
                retries -= 1    
                time.sleep(5)    
            else:    
                raise e  
            
    print('API is not responding, moving on...')   
    return None

In [444]:
# output = GPTextract("Software. I develop software for a living, so in my spare time and for my work environment, I want a software experience that is the most stable and pleasing. Apple is not perfect, but they are the closest to that")
# print(output)

In [445]:
# output = GPTextract("My car feels like it is a very muscular car and is driven manually")
# print(output)

In [446]:
# output = GPTextract("tifu by talking down apple products to my company is ceo. this literally happened a few minutes ago and i may be in need of searching for a new job pls halp i created a project at my work that allowed all of our products to be linked up to android apps that our customers can use to control them today was the day i was presenting this to upper management before starting my presentation the ceo of my company stopped in and decided he was interested in my project the presentation goes well and is very well received everyone thinks its a great idea to link our products up to apps instead of controlling them via switches and push buttons the final slide of my project was titled why android initially this is where things went very very bad i presented data showing that android based devices currently hold gt 50percent market share and apple is only around 38percent i then proceeded to tell everyone 82percent of new phones being shipped right now are android based i also explained that development for android was much easier than ios after i finish making these points my ceo just responds with well 82percent of people are stupid obviously thinking it was a joke and he just liked apple more i responded back with looks like we have a fanboy in the room the room went dead silent for the remainder of my presentation i finish up the presentation thinking it went quite well and the ceo storms out one of the vps in the meeting pulls me aside and politely fills me in on the fact that our ceo is one of the largest shareholders of apple i kinda just left work after that one  amp nbsp  tl dr shit on apple in front of my ceo who is one of the biggest investors in the company and called him a fucking fanboy edit rip inbox edit2 in the presentation i talked about expanding to ios when the time presented itself for now having the fully designed android app fits everyone is needs that is why the slide was titled why android initially  edit3 yes i own an android would i call myself a fan boy maybe i just prefer superior hardware for lower costs and open source whatever platform offers me that is the one i will choose i stated the android facts not to discredit apple but to show the usefulness of having the android app an ios app could not be created in near the time i made the android app because i would have had to learn other languages etc it was also just flat out not needed initially ")
# print(output)

In [447]:
# create emotion column
df['emotion'] = ""

In [448]:
#CHECKPOINT

last_processed_index = 0
done = 0

In [449]:
# Instead of iterating over the entire DataFrame, start from the last processed index
# This way, if the script fails, you can just run it again and it will continue from where it left off

# NO TIME OUT
for index, row in df.iterrows():
    if index <= last_processed_index:
        continue

    emotion = GPTextract(row['overall_text'])
    df.at[index, 'emotion'] = emotion

    last_processed_index = index  
    done += 1

    print(f'Rows processed: {done}, Index: {index}')


Rows processed: 1, Index: 1
Rows processed: 2, Index: 2
Rows processed: 3, Index: 3
Rows processed: 4, Index: 4
Rows processed: 5, Index: 5
Rows processed: 6, Index: 6
Rows processed: 7, Index: 7
Rows processed: 8, Index: 8
Rows processed: 9, Index: 9
Rows processed: 10, Index: 10
Rows processed: 11, Index: 11
Rows processed: 12, Index: 12
Rows processed: 13, Index: 13
Rows processed: 14, Index: 14
Rows processed: 15, Index: 15
Rows processed: 16, Index: 16
Rows processed: 17, Index: 17
Rows processed: 18, Index: 18
Rows processed: 19, Index: 19
Rows processed: 20, Index: 20
Rows processed: 21, Index: 21
Rows processed: 22, Index: 22
Rows processed: 23, Index: 23
Rows processed: 24, Index: 24
Rows processed: 25, Index: 25
Rows processed: 26, Index: 26
Rows processed: 27, Index: 27
Rows processed: 28, Index: 28
Rows processed: 29, Index: 29
Rows processed: 30, Index: 30
Rows processed: 31, Index: 31
Rows processed: 32, Index: 32
Rows processed: 33, Index: 33
Rows processed: 34, Index: 3

In [450]:
print('last_processed_index: ', last_processed_index)
print('done: ', done)

last_processed_index:  454
done:  454


In [451]:
print( df['emotion'].value_counts() )

Friendly                                                                                                                   134
Modern                                                                                                                      80
Elegant                                                                                                                     70
Dynamic                                                                                                                     59
Aggressive                                                                                                                  40
Practical                                                                                                                   22
Stable                                                                                                                      18
Powerful                                                                                                       

In [452]:
# count how many emotions is categorised under the keys list
df['emotion'].value_counts().loc[keys_list].sum()

412

In [453]:
# remove entries that are not in the keys list
df = df[df.emotion.isin(keys_list)]

len(df)

412

In [454]:
#calculate the percentage of each emotions in the dataframe
emotion_dict = df['emotion'].value_counts(normalize=True).to_dict()

emotion_dict

{'Friendly': 0.32524271844660196,
 'Modern': 0.1941747572815534,
 'Elegant': 0.16990291262135923,
 'Dynamic': 0.14320388349514562,
 'Aggressive': 0.0970873786407767,
 'Stable': 0.043689320388349516,
 'Powerful': 0.014563106796116505,
 'Sporty': 0.012135922330097087}

In [455]:
# Get the index of the first row
first_row_index = df.index[0]

# Store the dictionary in the first row of the 'score' column
df.loc[first_row_index, 'score'] = str(emotion_dict)

In [456]:
df.head(100)

,Unnamed: 0,subreddit,title,selftext,ups,overall_text,comment ups,sentiment,relevance,emotion,score
1,2,NaN,NaN,NaN,NaN,my daiso haul that i collected over the months...,6.0,POSITIVE,related to daiso,Friendly,"{'Friendly': 0.32524271844660196, 'Modern': 0...."
2,3,NaN,NaN,NaN,NaN,so so cute omg there is some stuff i have but ...,5.0,POSITIVE,related to daiso,Friendly,NaN
3,4,NaN,NaN,NaN,NaN,wow i had no idea daiso had so much of our girl,3.0,POSITIVE,related to daiso,Friendly,NaN
4,9,NaN,NaN,NaN,NaN,thank you so much glad you liked it hopefully ...,4.0,POSITIVE,related to daiso,Modern,NaN
5,10,NaN,NaN,NaN,NaN,i know righttt,1.0,POSITIVE,related to daiso,Dynamic,NaN
...,...,...,...,...,...,...,...,...,...,...,...
118,410,NaN,NaN,NaN,NaN,you welcome people who wrote that,3.0,POSITIVE,related to daiso,Friendly,NaN
119,415,NaN,NaN,NaN,NaN,all the more i have the urge to open the package,-1.0,POSITIVE,related to daiso,Aggressive,NaN
120,427,NaN,NaN,NaN,NaN,some people just want to watch the world burn ...,88.0,POSITIVE,related to daiso,Aggressive,NaN
121,432,NaN,NaN,NaN,NaN,the absolute instict in our malaysian blood to...,13.0,POSITIVE,related to daiso,Aggressive,NaN


In [457]:
# df.to_csv(r'/Users/raymondharrison/Desktop/DND/{}.csv'.format(search))

In [458]:
# df.to_csv(r'/Users/raymondharrison/Desktop/DND/{}.csv'.format(name))

# GPT API Emotional Color

In [459]:
# name = "LV review copy"
# df = pd.read_csv(f'/Users/raymondharrison/Desktop/DND/{name}.csv')

In [460]:
emotional_color_bag = {
    'Anger': ['rage', 'fury', 'irritation', 'wrath', 'hostility', 'anger'],
    'Balance': ['equilibrium', 'stability', 'harmony', 'evenness', 'poise', 'balance'],
    'Cold': ['chilly', 'frigid', 'aloof', 'unemotional', 'detached', 'cold'],
    'Courage': ['bravery', 'valor', 'fearlessness', 'boldness', 'heroism', 'courage'],
    'Danger': ['risk', 'peril', 'threat', 'hazard', 'menace', 'danger'],
    'Decadence': ['decline', 'degeneration', 'decadency', 'luxury', 'self-indulgence', 'decadence'],
    'Earthy': ['natural', 'unrefined', 'rustic', 'grounded', 'earthen', 'earthy'],
    'Energy': ['vitality', 'vigour', 'liveliness', 'dynamism', 'power', 'energy'],
    'Excitement': ['thrill', 'enthusiasm', 'euphoria', 'fervor', 'stimulation', 'excitement'],
    'Family': ['kinship', 'relatives', 'household', 'lineage', 'ancestry', 'family'],
    'Femininity': ['womanhood', 'gentleness', 'softness', 'modesty', 'delicacy', 'feminity'],
    'Flamboyance': ['extravagance', 'showiness', 'flair', 'ostentation', 'vividness', 'flamboyance'],
    'Freedom': ['liberty', 'independence', 'autonomy', 'unrestraint', 'free-will', 'freedom'],
    'Friendly': ['amicable', 'affable', 'genial', 'cordial', 'congenial', 'friendly'],
    'Fun': ['enjoyment', 'entertainment', 'amusement', 'pleasure', 'lightheartedness', 'fun'],
    'Fortune': ['fortune', 'serendipity', 'prosperity', 'favor', 'blessing'],
    'Growth': ['development', 'expansion', 'maturation', 'evolution', 'progression', 'growth'],
    'Happiness': ['joy', 'contentment', 'pleasure', 'cheerfulness', 'bliss', 'happiness'],
    'Intelligence': ['smart', 'brainy', 'knowledgeable', 'ingenious', 'perceptive', 'intelligence'],
    'Joy': ['delight', 'glee', 'happiness', 'jubilation', 'exultation', 'joy'],
    'Loyalty': ['faithfulness', 'allegiance', 'devotion', 'fidelity', 'steadfastness', 'loyalty'],
    'Luxury': ['opulence', 'lavishness', 'sumptuousness', 'extravagance', 'splendor', 'luxury'],
    'Modesty': ['humility', 'demureness', 'simplicity', 'shyness', 'moderation,modesty'],
    'Money': ['wealth', 'currency', 'capital', 'finances', 'treasure', 'money'],
    'Mystery': ['enigma', 'mystique', 'secrecy', 'puzzle', 'unknown', 'mystery'],
    'Nature': ['wildlife', 'environment', 'outdoors', 'flora', 'fauna', 'wildlife'],
    'Passion': ['ardor', 'zeal', 'enthusiasm', 'fervor', 'love', 'passion'],
    'Peace': ['tranquility', 'serenity', 'calmness', 'harmony', 'quietude', 'peace'],
    'Power': ['dominance', 'authority', 'control', 'influence', 'might', 'power'],
    'Personal-power': ['self-worth', 'confidence', 'autonomy', 'determination', 'influence', 'personal-power'],
    'Purity': ['purity', 'pure', 'innocence'],
    'Rational': ['logical', 'sensible', 'reasonable', 'analytical', 'coherent', 'rational'],
    'Reliable': ['dependable', 'trustworthy', 'consistent', 'faithful', 'staunch', 'reliable'],
    'Respect': ['esteem', 'regard', 'honor', 'reverence', 'admiration', 'respect'],
    'Royalty': ['sovereignty', 'monarchy', 'nobility', 'majesty', 'regal', 'royalty'],
    'Self-cultivation': ['self-improvement', 'personal growth', 'enlightenment', 'development', 'education', 'self-cultivation'],
    'Strength': ['fortitude', 'robustness', 'toughness', 'vigor', 'power', 'strength'],
    'Style': ['fashion', 'elegance', 'chic', 'grace', 'taste', 'style'],
    'Success': ['achievement', 'triumph', 'victory', 'prosperity', 'accomplishment', 'success'],
    'Trust': ['confidence', 'belief', 'faith', 'reliance', 'assurance', 'trust'],
    'Virtue': ['righteousness', 'goodness', 'integrity', 'morality', 'honesty', 'virtue'],
    'Warmth': ['heat', 'cordiality', 'passion', 'enthusiasm', 'affection', 'warmth'],
    'Wisdom': ['sagacity', 'knowledge', 'intelligence', 'judgment', 'insight', 'wisdom']
}


In [461]:
keys_list = list(emotional_color_bag.keys())
keys_string = ', '.join(keys_list)
print(keys_string)

Anger, Balance, Cold, Courage, Danger, Decadence, Earthy, Energy, Excitement, Family, Femininity, Flamboyance, Freedom, Friendly, Fun, Fortune, Growth, Happiness, Intelligence, Joy, Loyalty, Luxury, Modesty, Money, Mystery, Nature, Passion, Peace, Power, Personal-power, Purity, Rational, Reliable, Respect, Royalty, Self-cultivation, Strength, Style, Success, Trust, Virtue, Warmth, Wisdom


In [462]:
# https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api

import openai
import time

# Set OpenAI API key
openai.api_key = credentials.api_key

def GPTextract(prompt, model="gpt-3.5-turbo-1106", labels_bags=emotional_color_bag):

    retries = 3    
    while retries > 0:    
        try: 
            messages = [
                {"role": "system", "content": 
                f'''

                You are a powerful data analyst looking at labeling unstructured text semantically according to my bag of words. 
                {str(labels_bags)}

                You must output only one word from the bag of words(choose from {keys_string}). Dont need to give me an explanation.

                '''},
                {"role": "user", "content": prompt}
                ]
            response = openai.ChatCompletion.create(
                model=model, # gpt-3.5-turbo
                messages=messages, # prompt
                temperature=0, # temperature 0 means deterministic, 1 means random
            )
            text = response.choices[0].message["content"]
            text = text.replace("\n","")
            return text
        except Exception as e:    
            if e: 
                print(e)   
                print('Timeout error, retrying...')    
                retries -= 1    
                time.sleep(5)    
            else:    
                raise e  
            
    print('API is not responding, moving on...')   
    return None

In [463]:
# create emotional_color column
df['emotional_color'] = ""

In [464]:
#CHECKPOINT

last_processed_index = 0
done = 0

In [465]:
# Instead of iterating over the entire DataFrame, start from the last processed index
# This way, if the script fails, you can just run it again and it will continue from where it left off

# NO TIME OUT
for index, row in df.iterrows():
    if index <= last_processed_index:
        continue

    emotion = GPTextract(row['overall_text'])
    df.at[index, 'emotional_color'] = emotion

    last_processed_index = index  
    done += 1

    print(f'Rows processed: {done}, Index: {index}')


Rows processed: 1, Index: 1
Rows processed: 2, Index: 2
Rows processed: 3, Index: 3
Rows processed: 4, Index: 4
Rows processed: 5, Index: 5
Rows processed: 6, Index: 6
Rows processed: 7, Index: 7
Rows processed: 8, Index: 8
Rows processed: 9, Index: 9
Rows processed: 10, Index: 10
Rows processed: 11, Index: 11
Rows processed: 12, Index: 14
Rows processed: 13, Index: 15
Rows processed: 14, Index: 16
Rows processed: 15, Index: 17
Rows processed: 16, Index: 18
Rows processed: 17, Index: 19
Rows processed: 18, Index: 20
Rows processed: 19, Index: 21
Rows processed: 20, Index: 22
Rows processed: 21, Index: 23
Rows processed: 22, Index: 28
Rows processed: 23, Index: 29
Rows processed: 24, Index: 30
Rows processed: 25, Index: 33
Rows processed: 26, Index: 34
Rows processed: 27, Index: 35
Rows processed: 28, Index: 36
Rows processed: 29, Index: 39
Rows processed: 30, Index: 40
Rows processed: 31, Index: 41
Rows processed: 32, Index: 42
Rows processed: 33, Index: 43
Rows processed: 34, Index: 4

In [466]:
print('last_processed_index: ', last_processed_index)
print('done: ', done)

last_processed_index:  454
done:  412


In [467]:
print( df['emotional_color'].value_counts() )

Excitement                                                                                          98
Fun                                                                                                 38
Family                                                                                              26
Anger                                                                                               21
Balance                                                                                             18
                                                                                                    ..
Patience                                                                                             1
Peace                                                                                                1
Sorry, I'm not sure I understand. Could you please provide more context or clarify your request?     1
Rational                                                                 

In [468]:
# # count how many emotions is categorised under the keys list
# df['emotional_color'].value_counts().loc[keys_list].sum()

In [469]:
# remove entries that are not in the keys list
df = df[df.emotional_color.isin(keys_list)]

len(df)

375

In [470]:
#calculate the percentage of each emotions in the dataframe
emotional_color_dict = df['emotional_color'].value_counts(normalize=True).to_dict()

emotional_color_dict

{'Excitement': 0.2613333333333333,
 'Fun': 0.10133333333333333,
 'Family': 0.06933333333333333,
 'Anger': 0.056,
 'Balance': 0.048,
 'Femininity': 0.04,
 'Nature': 0.037333333333333336,
 'Joy': 0.037333333333333336,
 'Flamboyance': 0.034666666666666665,
 'Happiness': 0.034666666666666665,
 'Friendly': 0.032,
 'Success': 0.029333333333333333,
 'Courage': 0.029333333333333333,
 'Style': 0.02666666666666667,
 'Warmth': 0.02666666666666667,
 'Respect': 0.024,
 'Growth': 0.010666666666666666,
 'Danger': 0.010666666666666666,
 'Earthy': 0.010666666666666666,
 'Trust': 0.010666666666666666,
 'Freedom': 0.008,
 'Passion': 0.008,
 'Mystery': 0.008,
 'Cold': 0.008,
 'Modesty': 0.008,
 'Fortune': 0.008,
 'Decadence': 0.005333333333333333,
 'Money': 0.0026666666666666666,
 'Peace': 0.0026666666666666666,
 'Rational': 0.0026666666666666666,
 'Reliable': 0.0026666666666666666,
 'Intelligence': 0.0026666666666666666,
 'Loyalty': 0.0026666666666666666}

In [471]:
# Get the index of the first row
first_row_index = df.index[0]

# Store the dictionary in the first row of the 'score' column
df.loc[first_row_index, 'emotional color score'] = str(emotional_color_dict)

In [472]:
df.head(100)

,Unnamed: 0,subreddit,title,selftext,ups,overall_text,comment ups,sentiment,relevance,emotion,score,emotional_color,emotional color score
1,2,NaN,NaN,NaN,NaN,my daiso haul that i collected over the months...,6.0,POSITIVE,related to daiso,Friendly,"{'Friendly': 0.32524271844660196, 'Modern': 0....",Excitement,"{'Excitement': 0.2613333333333333, 'Fun': 0.10..."
2,3,NaN,NaN,NaN,NaN,so so cute omg there is some stuff i have but ...,5.0,POSITIVE,related to daiso,Friendly,NaN,Excitement,NaN
3,4,NaN,NaN,NaN,NaN,wow i had no idea daiso had so much of our girl,3.0,POSITIVE,related to daiso,Friendly,NaN,Excitement,NaN
4,9,NaN,NaN,NaN,NaN,thank you so much glad you liked it hopefully ...,4.0,POSITIVE,related to daiso,Modern,NaN,Excitement,NaN
5,10,NaN,NaN,NaN,NaN,i know righttt,1.0,POSITIVE,related to daiso,Dynamic,NaN,Excitement,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,473,NaN,NaN,NaN,NaN,pretty cool will be sure to get some 1 144 1 100,15.0,POSITIVE,related to daiso,Modern,NaN,Excitement,NaN
134,474,NaN,NaN,NaN,NaN,i hope my daiso have them,10.0,POSITIVE,related to daiso,Modern,NaN,Balance,NaN
137,496,NaN,NaN,NaN,NaN,i would pay four times that very happily i dre...,1.0,POSITIVE,related to daiso,Elegant,NaN,Fortune,NaN
138,499,NaN,NaN,NaN,NaN,hi does ion orchard still have the stand,1.0,POSITIVE,related to daiso,Modern,NaN,Style,NaN


In [473]:
df.to_csv(r'/Users/raymondharrison/Desktop/DND/{}.csv'.format(name))